# 📊📚 Analysez des données de systèmes éducatifs

## Sommaire
[Introduction](#Introduction)  

[1. Librairies et chargement des data](#1.-Librairies-et-chargement-des-data)
- [a. Librairies importées](#a.-Librairies-importées)
- [b. Chargement des fichiers CSV](#b.-Chargement-des-fichiers-CSV)

[2. Nettoyage des dataframes](#2.-Nettoyage-des-dataframes)
- [a. Création du log df](#a.-Création-du-log-df)
- [b. Suppression des colonnes avec na_rate = 1](#b.-Suppression-des-colonnes-avec-na_rate-=-1)
- [c. Suppression des faux pays](#c.-Suppression-des-faux-pays)

## Introduction

Academy est une plateforme d'éducation en ligne pour les étudiants du niveau lycée et université.
La start-up qui a développé la plateforme souhaite l'étendre à un niveau internationnal. 

Pour ce faire, il faut dans un premier temps **s'assurer que les données récoltées de la banque mondiale sont exploitables** pour enrichir la réflexion. 
Puis, dans un second temps, **identifier les pays cibles avec un fort potentiel** et voir leur évolution dans le temps.

### 1. Librairies et chargement des data

#### a. Librairies importées

In [269]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

#### b. Chargement des fichiers CSV

In [270]:
df_data = pd.read_csv('../dataset/EdStatsData.csv')
df_country_series = pd.read_csv('../dataset/EdStatsCountry-Series.csv')
df_country = pd.read_csv('../dataset/EdStatsCountry.csv')
df_foot_note = pd.read_csv('../dataset/EdStatsFootNote.csv')
df_stats_series = pd.read_csv('../dataset/EdStatsSeries.csv')

###### Description des DF

**df_data** : Ensemble des indicateurs par pays et par années (1970 à 2100). Chaque ligne représente le couple pays/indicateurs.

**df_country** : Liste d'informations complémentaires concernant les pays. Chaque ligne correspond à un pays.

**df_country_series** : Indicateurs démographiques et économiques par année et par pays présents dans df_data. Chaque ligne repésente le couples pays/indicateurs/années.

**df_stats_series** : Lexique des indicateurs. Chaque lique correspond à un indicateur.

**df_foot_note** : Informations additionnelles sur les indicateurs basées sur l'année, le type d'indicateur et le pays dans df_data.

In [271]:
#add widget to display df.head(5)

### 2. Nettoyage des dataframes

#### a. Création du log df 

In [272]:
#Log df infos
dic = {"df_data" : df_data, "df_country":df_country, "df_country_series" : df_country_series, 
           "df_foot_note" : df_foot_note, "df_stats_series" : df_stats_series}


cat_log = ['shape','duplicated_rows','na_rate', 'stats','nunique_cat']

obj_log = {
    'log_cat': cat_log,
    "df_data": [None] * len(cat_log),
    "df_country": [None] * len(cat_log),
    "df_country_series": [None] * len(cat_log),
    "df_foot_note": [None] * len(cat_log),
    "df_stats_series": [None] * len(cat_log)
}

df_log = pd.DataFrame(obj_log)

In [273]:
dic_keys = list(dic.keys())

for df_name in dic_keys:
    
    #Shape
    shape_value = dic[df_name].shape
    df_log.loc[df_log['log_cat'] == 'shape', df_name] = [shape_value]
    
    #duplicated_rows
    duplicates = dic[df_name].duplicated().sum()
    df_log.loc[df_log['log_cat'] == 'duplicated_rows', df_name] = [duplicates]
    
    #na_rate
    na_rate = round(dic[df_name].isna().sum() / len(dic[df_name]),2)
    df_log.loc[df_log['log_cat'] == 'na_rate', df_name] = [na_rate]
    
    #stats df.describe(include=[np.number])
    stats_report = dic[df_name].describe(include=[np.number])
    df_log.loc[df_log['log_cat'] == 'stats', df_name] = [stats_report]
    
    #Nb uniques categories df_obj = df.select_dtypes(include='object')
    unique_categories = dic[df_name].select_dtypes(include='object').nunique()
    df_log.loc[df_log['log_cat'] == 'nunique_cat', df_name] = [unique_categories]

In [274]:
df_log

,log_cat,df_data,df_country,df_country_series,df_foot_note,df_stats_series
0,shape,"(886930, 70)","(241, 32)","(613, 4)","(643638, 5)","(3665, 21)"
1,duplicated_rows,0,0,0,0,0
2,na_rate,Country Name 0.00 Country Code 0.00 ...,Country Code ...,CountryCode 0.0 SeriesCode 0.0 DESCRIPT...,CountryCode 0.0 SeriesCode 0.0 Year ...,Series Code 0.00 To...
3,stats,1970 1971 197...,National accounts reference year Lates...,Unnamed: 3 count 0.0 mean ...,Unnamed: 4 count 0.0 mean ...,Unit of measure Notes from original so...
4,nunique_cat,Country Name 242 Country Code 242 ...,Country Code ...,CountryCode 211 SeriesCode 21 DESCRIPT...,CountryCode 239 SeriesCode 1558 Year ...,Series Code 3665 To...


In [275]:
# Widgets : Toggle pour log_cat et Toggle pour les colonnes
toggle_cat = widgets.ToggleButtons(
    options=df_log['log_cat'].values,
    description='Category:',
    button_style=''
)

toggle_df = widgets.ToggleButtons(
    options=df_log.columns[1:],
    description='Dataframe:',
    button_style=''
)

# Output widget pour afficher le résultat
out = widgets.Output()

# Callback qui combine les deux sélections
def update_value(change):
    category_name = toggle_cat.value
    df_name = toggle_df.value
    
    with out:
        out.clear_output()
        try:
            valeur = df_log.loc[df_log['log_cat'] == category_name, df_name].iloc[0]
            
            if isinstance(valeur, pd.Series):
                display(valeur.to_frame())
            else:
                display(valeur)
                
        except Exception as e:
            print("Erreur :", e)

# Observer les changements sur les deux widgets
toggle_cat.observe(update_value, names='value')
toggle_df.observe(update_value, names='value')

# Affichage
display(toggle_cat, toggle_df, out)

ToggleButtons(description='Category:', options=('shape', 'duplicated_rows', 'na_rate', 'stats', 'nunique_cat')…

ToggleButtons(description='Dataframe:', options=('df_data', 'df_country', 'df_country_series', 'df_foot_note',…

Output()

#### b. Suppression des colonnes avec na_rate = 1

In [276]:
if df_data.loc[:,['Country Name','Indicator Name']].duplicated().sum() == 0:
    print('✅ Pas de doublon pour le couple Pays/Indicateurs dans le df_data')

✅ Pas de doublon pour le couple Pays/Indicateurs dans le df_data


In [277]:
col_to_delete = []

for df_name in dic_keys:
    temp_df = df_log.loc[df_log['log_cat'] == "na_rate", df_name].iloc[0].reset_index()
    
    #col name
    new_col_name = ['col_name','na_rate']
    temp_df.columns = new_col_name
    temp_df_filtered = temp_df[temp_df['na_rate'] == 1]
    col_list = temp_df_filtered['col_name'].astype(str).tolist()
    
    for col in col_list:
        col_to_delete.append(col)

print(col_to_delete)

['2017', 'Unnamed: 69', 'Unnamed: 31', 'Unnamed: 3', 'Unnamed: 4', 'Unit of measure', 'Limitations and exceptions', 'Notes from original source', 'General comments', 'Development relevance', 'Other web links', 'Related indicators', 'License Type', 'Unnamed: 20']


In [278]:
col_to_delete_set = set(col_to_delete)

for df_name in dic_keys:
    col_to_keep = [x for x in dic[df_name].columns if x not in col_to_delete_set]
    dic[df_name] = dic[df_name][col_to_keep]

### c. Suppression des faux pays

In [279]:
#from df_country['Short Name'].unique()
region_to_delete = ['Arab World','East Asia & Pacific (developing only)',
       'East Asia & Pacific (all income levels)',
       'Europe & Central Asia (developing only)',
       'Europe & Central Asia (all income levels)','Euro area','European Union','High income',
       'Heavily indebted poor countries (HIPC)','Latin America & Caribbean (developing only)',
       'Latin America & Caribbean (all income levels)',
       'Least developed countries: UN classification', 'Low income','Lower middle income',
       'Low & middle income','Middle East & North Africa (all income levels)','Middle income',
       'OECD members','Sub-Saharan Africa (developing only)','Sub-Saharan Africa (all income levels)',
       'Middle East & North Africa (developing only)',
       'World','Latin America & Caribbean',
       'Middle East & North Africa (excluding high income)',
       'North America', 'South Asia', 'Upper middle income',]

In [280]:
df_region = df_country[df_country['Short Name'].isin(region_to_delete)]
df_region = df_region.loc[:,['Short Name','Country Code']]

In [281]:
dic['df_country_series'] = dic['df_country_series'].rename(columns={'CountryCode' : 'Country Code'})
dic['df_foot_note'] = dic['df_foot_note'].rename(columns={'CountryCode' : 'Country Code'})

In [282]:
for df_name in dic_keys:
    if df_name != 'df_stats_series':
        dic[df_name] = dic[df_name][~dic[df_name]['Country Code'].isin(df_region['Country Code'].unique())]

## 3. Sélection des indicateurs et filtrage du df data

#### a. Indicateurs

#### Critères de recherches. 

il faudrait que les pays :

 - aient une accessibilité restreinte du niveau secondaire et universitaire. Soit par:
     - un manque d'infrasturcture
     - un manque de personnel
     - réservé à une certaine population (classe sociale, genre...).
 - aient une population jeune avec une demande forte pour léducation (% de diplômé faible par rapport à la population en âge de l'être).
 - soient un minimum développé de sorte à ce que les étudiants puissent se procurer le matériel informatique adéquat.

**Pourquoi ?** 

✅ Opportunité de proposer une alternative.

✅ Cette proposition est viable s'il y a de la demande.

✅ Et cette dernière doit avoir les moyens d'accéder au produit.

In [284]:
# Identification des indicateurs
dic_kpis = {
    'School accessibility': {
        'Teachers': [
            'Teachers in secondary education, both sexes (number)',
            'Teachers in tertiary education programmes, both sexes (number)',
        ],
        'Education Management Information Systems (SABER)': [
            'SABER: (Education Management Information Systems) Policy Goal 1 Lever 4: Infrastructural capacity'
        ],
        'Attainment': [
            'Barro-Lee: Percentage of population age 15+ with no education'
        ],
        'Education Equality': [
            #multiple data sources
            'MICS: Secondary completion rate. Female',
            'MICS: Secondary completion rate. Male',
            'MICS: Secondary completion rate. Rural',
            'MICS: Secondary completion rate. Urban',
            'DHS: Primary completion rate. Female',
            'DHS: Primary completion rate. Male',
            'DHS: Primary completion rate. Rural',
            'DHS: Primary completion rate. Urban',
            'UIS: Total net attendance rate, lower secondary, female (%)',
            'UIS: Total net attendance rate, lower secondary, male (%)',
            'UIS: Total net attendance rate, lower secondary, rural, both sexes (%)',
            'UIS: Total net attendance rate, lower secondary, urban, both sexes (%)', 
        ]
    },
    'Demography': {
        'Population': [
            'Population of the official age for secondary education, both sexes (number)'
        ],
        'Health: Population: Structure': [
            'Population, total'
        ],
        'Health: Population: Dynamics': [
            'Population growth (annual %)'
        ]
    },
    'Economic': {
        'Infrastructure: Communications': [
            'Personal computers (per 100 people)',
            'Internet users (per 100 people)'
        ]
    }
}

#### b. Filtrage du df data

In [285]:
kpis_list = []
for theme in dic_kpis:
    for topic in dic_kpis[theme]:
        for kpi in dic_kpis[theme][topic]:
            kpis_list.append(kpi)

df_data_filtered = dic['df_data'][dic['df_data']['Indicator Name'].isin(kpis_list)]

#### c. Sélection des années

In [292]:
years_to_keep = ['2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2020', '2025', '2030']
df_data_filtered_years = df_data_filtered.loc[:,['Country Name','Country Code','Indicator Name','Indicator Code'] + years_to_keep]

In [295]:
df_data_filtered_years.groupby(['Country Name','Indicator Name'])[[years_to_keep]].apply(lamba g: g.isna().sum())

SyntaxError: invalid syntax (2161565088.py, line 1)

In [304]:
df_data_filtered_years.groupby(
    ['Indicator Name']
)[years_to_keep].apply(lambda g: g.isna().mean()).reset_index().transpose()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Indicator Name,Barro-Lee: Percentage of population age 15+ wi...,DHS: Primary completion rate. Female,DHS: Primary completion rate. Male,DHS: Primary completion rate. Rural,DHS: Primary completion rate. Urban,Internet users (per 100 people),MICS: Secondary completion rate. Female,MICS: Secondary completion rate. Male,MICS: Secondary completion rate. Rural,MICS: Secondary completion rate. Urban,...,Population growth (annual %),Population of the official age for secondary e...,"Population, total",SABER: (Education Management Information Syste...,"Teachers in secondary education, both sexes (n...","Teachers in tertiary education programmes, bot...","UIS: Total net attendance rate, lower secondar...","UIS: Total net attendance rate, lower secondar...","UIS: Total net attendance rate, lower secondar...","UIS: Total net attendance rate, lower secondar..."
2000,0.336406,0.953917,0.953917,0.953917,0.953917,0.096774,1.0,1.0,1.0,1.0,...,0.009217,0.124424,0.009217,1.0,0.474654,0.497696,0.990783,0.990783,0.990783,0.990783
2001,1.0,0.976959,0.976959,0.976959,0.976959,0.092166,1.0,1.0,1.0,1.0,...,0.009217,0.115207,0.009217,1.0,0.470046,0.525346,0.995392,0.995392,0.995392,0.995392
2002,1.0,0.995392,0.995392,0.995392,0.995392,0.082949,1.0,1.0,1.0,1.0,...,0.009217,0.105991,0.009217,1.0,0.488479,0.516129,1.0,1.0,1.0,1.0
2003,1.0,0.953917,0.953917,0.953917,0.953917,0.110599,1.0,1.0,1.0,1.0,...,0.009217,0.105991,0.009217,1.0,0.488479,0.543779,0.97235,0.97235,0.97235,0.97235
2004,1.0,0.976959,0.976959,0.976959,0.976959,0.096774,1.0,1.0,1.0,1.0,...,0.009217,0.105991,0.009217,1.0,0.483871,0.539171,0.967742,0.967742,0.967742,0.967742
2005,0.336406,0.935484,0.935484,0.935484,0.935484,0.087558,0.963134,0.963134,0.963134,0.963134,...,0.009217,0.105991,0.009217,1.0,0.497696,0.580645,0.9447,0.9447,0.9447,0.9447
2006,1.0,0.963134,0.963134,0.963134,0.963134,0.092166,0.963134,0.963134,0.97235,0.97235,...,0.009217,0.105991,0.009217,1.0,0.506912,0.576037,0.870968,0.870968,0.870968,0.870968
2007,1.0,0.97235,0.97235,0.97235,0.97235,0.059908,0.995392,0.995392,0.995392,0.995392,...,0.009217,0.101382,0.009217,1.0,0.474654,0.576037,0.9447,0.9447,0.9447,0.9447
2008,1.0,0.958525,0.958525,0.958525,0.958525,0.064516,1.0,1.0,1.0,1.0,...,0.009217,0.101382,0.009217,1.0,0.520737,0.580645,0.967742,0.967742,0.967742,0.967742
